In [3]:
import os, glob, json

import numpy as np
import pandas as pd

import keras

from keras.models import Sequential, Model
from keras.layers import LSTM, Input
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Conjunto de teste

In [7]:
dataset = pd.read_pickle(r'../data/trainset/dataset.pkl')

In [8]:
X = np.array([[list(y) for y in list(x)] for x in dataset.features.values])
#X = dataset.features.values
y = dataset.label.values
y = to_categorical(y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X.shape

(1192, 23, 130)

In [11]:
y[0]

array([0., 1., 0.], dtype=float32)

In [12]:
y_test

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [13]:
INPUT_SHAPE = (23,130)

Modelos

In [14]:
class BiLTSM():
    def __init__(self, n_hidden_layers, units_per_layer, dropout=0.5):
        self.n_hidden_layers = n_hidden_layers
        self.units_per_layer = units_per_layer
        self.dropout = dropout
    
    def build_model(self):
        metrcs = ['accuracy']
        '''
        self.model = Sequential();
        for i in range(self.n_hidden_layers-1):
            self.model.add(Bidirectional(LSTM(units=self.units_per_layer,
                        input_shape=INPUT_SHAPE,
                      return_sequences=False)))
        self.model.add(Bidirectional(LSTM(self.units_per_layer,
                        input_shape=INPUT_SHAPE,
                      return_sequences=False)))
        self.model.add(Dense(3, activation='softmax'))
        '''
        inp = Input(shape=(23, 130))
        x = Bidirectional(LSTM(120, return_sequences=True))(inp)
        x = Bidirectional(LSTM(120, return_sequences=True))(x)
        x = Bidirectional(LSTM(120, return_sequences=False))(x)
        x = Dense(3, activation='softmax')(x)
        
        self.model = Model(inp, x)

        #self.model.summary()
        self.model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(), metrics=metrcs)
        
    def fit_model(self, epochs):
        self.model.fit(X_train, y_train, 
                       validation_split=.3, 
                       epochs=epochs, batch_size=32)

In [15]:
model1 = BiLTSM(1, 120)

In [16]:
model1.build_model()

In [17]:
model1.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23, 130)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 23, 240)           240960    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 23, 240)           346560    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 240)               346560    
_________________________________________________________________
dense (Dense)                (None, 3)                 723       
Total params: 934,803
Trainable params: 934,803
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.fit_model(100)

Epoch 1/100
18/18 [==============================] - 2s 124ms/step - loss: 0.9463 - accuracy: 0.5125 - val_loss: 0.7326 - val_accuracy: 0.6417
Epoch 2/100
18/18 [==============================] - 1s 52ms/step - loss: 0.6298 - accuracy: 0.6953 - val_loss: 0.7131 - val_accuracy: 0.6458
Epoch 3/100
18/18 [==============================] - 1s 53ms/step - loss: 0.5098 - accuracy: 0.7581 - val_loss: 0.5336 - val_accuracy: 0.7958
Epoch 4/100
18/18 [==============================] - 1s 53ms/step - loss: 0.4729 - accuracy: 0.7437 - val_loss: 0.4945 - val_accuracy: 0.7375
Epoch 5/100
18/18 [==============================] - 1s 53ms/step - loss: 0.3820 - accuracy: 0.8297 - val_loss: 0.6407 - val_accuracy: 0.7167
Epoch 6/100
18/18 [==============================] - 1s 53ms/step - loss: 0.3894 - accuracy: 0.8047 - val_loss: 0.3768 - val_accuracy: 0.8208
Epoch 7/100
18/18 [==============================] - 1s 52ms/step - loss: 0.3036 - accuracy: 0.8674 - val_loss: 0.3444 - val_accuracy: 0.8333
Epoch

18/18 [==============================] - 1s 51ms/step - loss: 0.0224 - accuracy: 0.9946 - val_loss: 0.4293 - val_accuracy: 0.9000
Epoch 59/100
18/18 [==============================] - 1s 57ms/step - loss: 0.0201 - accuracy: 0.9946 - val_loss: 0.4830 - val_accuracy: 0.9042
Epoch 60/100
18/18 [==============================] - 1s 55ms/step - loss: 0.0093 - accuracy: 0.9964 - val_loss: 0.4723 - val_accuracy: 0.9042
Epoch 61/100
18/18 [==============================] - 1s 55ms/step - loss: 0.0127 - accuracy: 0.9964 - val_loss: 0.4844 - val_accuracy: 0.9083
Epoch 62/100
18/18 [==============================] - 1s 57ms/step - loss: 0.0173 - accuracy: 0.9928 - val_loss: 0.4927 - val_accuracy: 0.9042
Epoch 63/100
18/18 [==============================] - 1s 57ms/step - loss: 0.0406 - accuracy: 0.9857 - val_loss: 0.5367 - val_accuracy: 0.9000
Epoch 64/100
18/18 [==============================] - 1s 57ms/step - loss: 0.0497 - accuracy: 0.9910 - val_loss: 0.5167 - val_accuracy: 0.8917
Epoch 65/100

In [ ]:
y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model1.model.predict(X_test, verbose=1)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))